# Name Merge Test

#### This notebook is aimed at testing the name merge performance between different dataset:
1. Burning Glass merge to Shark Repellent (BG is the query list, SR is the search dictionary)
- Shark Repellent merge to Compustat (SR is the query list, Comp is the search dictionary)
- Burning Glass merge to Compustat (BG is the query list, Comp is the search dictionary)

In [1]:
import pandas as pd
import numpy as np
import jieba
import difflib
import random
from cleanco import cleanco
import jaro
import string

 #### Download Data

In [2]:
comp=pd.read_stata(r"..\Compustat\names.dta")
shark=pd.read_excel(r"..\FactSet SharkRepllent Data (Pulled 2019-11-19).xlsx").drop(index=[0,1,2,4])
shark.columns=shark.loc[3]
shark=shark.drop(index=3).reset_index().drop('index',axis=1)
bg=pd.read_stata(r"..\00_BGT_Firm_Names.dta")
#create name
c_name=pd.DataFrame(comp['conm']).astype(str)
s_name=pd.DataFrame(shark.iloc[:,7]).rename({'Company Name':'conm'}, axis=1)
bg_name=pd.DataFrame(bg.name_bgt[bg['total_postings_bgt']>50])
bg_name_full=pd.DataFrame(bg.name_bgt)

#### Provide cleaned versions of names
1. 
 `cleanco` processes company names, providing cleaned versions of the names by stripping away terms indicating organization type (such as "Ltd." or "Corp").  
- Using a database of organization type terms, It also provides an utility to deduce the type of organization, in terms of US/UK business entity types (ie. "limited liability company" or "non-profit"). 

- Details about this package can be found at https://pypi.org/project/cleanco/

- I also change uppercase letter to lowercase.


In [3]:
#clean name
#remove organization type and thansfer to lower case
s_name1={}.fromkeys(list(map(lambda x: cleanco(x.lower()).clean_name(), s_name.conm))).keys()
c_name1={}.fromkeys(list(map(lambda x: cleanco(x.lower()).clean_name(), c_name.conm))).keys()
bg_name1={}.fromkeys(list(map(lambda x: cleanco(x.lower()).clean_name(), bg_name.name_bgt))).keys()

In [4]:
def jaro_distance(list1,list2):
    """
    Measure strings similarity by Jaro-winker distance.
    
    Parameters
    ----------
        list1: list of query strings
        list2: list of names dictionary       
    Returns
    -------
        df: Dataframe with three columns: "query_name", "match" and "score"
        "query_name" is the query string(target company name)
        "match" is the most similary string found at dictionary for the query string
        "score" is a float number used to measure the similarity between the query string and "match" string, range(0,1)
              
    """
    df=pd.DataFrame(list1)
    label=[]
    score_get=np.empty(len(list1))
    score=np.empty(len(list2))
    for n1,str1 in enumerate(list1):
        for n2,str2 in enumerate(list2):
            score[n2]=jaro.jaro_winkler_metric(str1,str2)
        imax=np.argmax(score)
        label.append(list2[imax])
        score_get[n1]=max(score)
    df['match']=label
    df['score']=score_get
    df.rename(columns={0:'query_name'})
    return df

In [5]:
class StrSimilarity1():
    def __init__(self,word):
        self.word=word
#Compared函数，参数str_list是对比字符串列表
#返回原始字符串分词后和对比字符串的匹配次数，返回一个字典
    def Compared(self,str_list):
        """
        Count common words.
        
        Parameters
        ----------
        str_list: a list contains potential matched strings
        
        
        Returns
        ----------
        dict_data: a dictionary
        keys are the potential matched strings
        value are the number of common words
        
        """
        dict_data={}
        sarticiple=list(self.word.strip().translate(str.maketrans('', '', string.punctuation)).split())
        for strs in str_list:
            #s_name list
            strs_word=list(strs.strip().translate(str.maketrans('', '', string.punctuation)).split())
            num=0
            for strs1 in strs_word:
                if strs1 in sarticiple:
                    num = num+1
                else: 
                    num = num
            dict_data[strs]=num
        return dict_data
    #NumChecks函数，参数dict_data是原始字符串分词后和对比字符串的匹配次数的字典，也就是Compared函数的返回值
    #返回出现次数最高的两个，返回一个字典
    def NumChecks(self,dict_data):
        """
        Return two potential strings with the hightest common word number.
        
        
        Parameters
        ----------
        dict_data: a dictionary
        keys are the potential matched strings
        value are the number of common words(return of Compared)
        
        
        Returns
        ----------
        dict_data: a dictionary
        keys are the two potential matched strings with the highest number of common words
        value are the number of common words
        
        """       
        list_data = sorted(dict_data.items(), key=lambda asd:asd[1], reverse=True)
        length = len(list_data)
        json_data = {}
        if length>=2:
            datas = list_data[:2]
        else:
            datas = list_data[:length]
        for data in datas:
            json_data[data[0]]=data[1]
        return json_data
#MMedian函数，参数dict_data是出现次数最高的两个对比字符串的字典，也就是NumChecks函数的返回值
#返回对比字符串和调节值的字典
    def MMedian(self,dict_data):
        """
        Calculate adjusted similarity scores for most potential strings(optional step).
        
        
        Parameters
        ----------
        dict_data: a dictionary
        keys are the two potential matched strings with the highest number of common words
        value are the number of common words(return of NumChecks)
        
        
        Returns
        ----------
        dict_data: a dictionary
        keys are the two potential matched strings with the highest number of common words
        value are the adjusted similarity scores
               
        """   
        
        median_list={}
        l=len(list(self.word.strip().translate(str.maketrans('', '', string.punctuation)).split()))#query string word numbers
        for k,v in dict_data.items():#k is potential string, v is the common word number
            length=len(list(k.strip().translate(str.maketrans('', '', string.punctuation)).split()))#potential string word numbers
            if l>v: 
                if v==length:
                    xx=-1
                else: 
                    xx = ((abs(l-v))/l)
            else: 
                 xx=-2    
            median_list[k] = xx
        return median_list
    
    
    
#Appear函数，参数dict_data是对比字符串和调节值的字典，也就是MMedian函数的返回值
#返回最相似的字符串
    def Appear(self,dict_data):
        """
        Return the most similar potential string.
        
        
        Parameters
        ----------
        dict_data: a dictionary
        keys are the two potential matched strings with the highest number of common words
        value are the adjusted similarity scores(return of  MMedian)
        
        
        Returns
        ----------
        dict_data: a dictionary
        key is the query string
        value is most similar potential string
               
        """   
        json_data={}
        for k,v in dict_data.items():
            fraction = difflib.SequenceMatcher(None, self.word, k).quick_ratio()-v
            json_data[k]=fraction
        tulp_data = sorted(json_data.items(), key=lambda asd:asd[1], reverse=True)
        return tulp_data[0][0],tulp_data[0][1]   
    
def name_match1(query_list1,str_list1):
    """
    Measure strings similarity by StrSimilary.
    
    Parameters
    ----------
        query_list1: list of query strings
        str_list1: list of names dictionary       
    Returns
    -------
        df: Dataframe with three columns: "query_name", "match" and "score"
        "query_name" is the query string(target company name)
        "match" is the most similary string found at dictionary for the query string
        "score" is a float number used to measure the similarity between the query string and "match" string, range(0,1)             
    """ 
    name_match=[]
    score=[]
    #str_list1=list(' '.join(str1.strip().translate(str.maketrans('', '', string.punctuation)).split()) for str1 in str_list1)
    for i,str_query in enumerate(query_list1):
        def main():
            query_str =str_query
            str_list=str_list1
    
            ss = StrSimilarity1(query_str)
            list_data = ss.Compared(str_list)
            num = ss.NumChecks(list_data)
            mmedian = ss.MMedian(num)
            #print(query_str,ss.Appear(mmedian))
            return ss.Appear(mmedian)
        if __name__=="__main__":
            name_match.append(main()[0])
            score.append(main()[1])
    df=pd.DataFrame(query_list1)
    df['match']=name_match
    df['score']=score
    df.rename(columns={0:'query_name'})
    return df

#### 1. Burning Glass —>Shark Repellent

In [69]:
#real data test
random.seed(123)
query_bg=random.sample(bg_name1,100)#1000 names 
s_name1=sorted(s_name1, key=len)    
dict_s=list(' '.join(str1.strip().translate(str.maketrans('', '', string.punctuation)).split()) for str1 in s_name1)
query_bg=list(' '.join(str1.strip().translate(str.maketrans('', '', string.punctuation)).split()) for str1 in query_bg)
query_bg.sort(reverse=False)

#### 1.1 Merge Result

- StrSimilar

In [71]:
%%time
df_bg2s=name_match1(query_bg,dict_s).sort_values(by='score',ascending=False)

Wall time: 8.87 s


In [72]:
df_bg2s[df_bg2s.score>0]

,0,match,score
46,it solutions,pomeroy it solutions,2.750000
97,triad,triad hospitals,2.500000
91,team sewell,team,1.533333
66,northwestern connecticut community college,northwestern,1.444444
75,rescare residential services,rescare,1.400000
86,spn well services,us well services,0.575758
37,gm financial group,meta financial group,0.561404
59,mb global logistics,echo global logistics,0.516667
36,global commerce services,premiere global services,0.500000
82,senior resource group,asa resource group,0.487179


In [73]:
#threshold=1
df_bg2s[df_bg2s.score>1]

,0,match,score
46,it solutions,pomeroy it solutions,2.750000
97,triad,triad hospitals,2.500000
91,team sewell,team,1.533333
66,northwestern connecticut community college,northwestern,1.444444
75,rescare residential services,rescare,1.400000


- Jaro-winkler

In [76]:
%%time
df_bg2s_j=jaro_distance(query_bg,dict_s).sort_values(by='score',ascending=False)

Wall time: 33.7 s


In [79]:
#threshold=0.8
df_bg2s_j[df_bg2s_j.score>0.8]

,0,match,score
86,spn well services,us well services,0.939951
43,hub international,sfbc international,0.905229
47,jefferson parish,jefferson bancshares,0.900714
20,catapult technology,datapulse technology,0.895304
72,pinnacle marketing,pinnacle airlines,0.891690
...,...,...,...
39,hamadeh educational services,hallmark financial services,0.814674
6,allfast fastening systems,alliance data systems,0.810593
29,escreen,rescare,0.809524
52,ledic management group,lee metal group,0.809062


In [80]:
#threshold=0.8
df_jaro[df_jaro.score>0.85]

,0,match,score
86,spn well services,us well services,0.939951
43,hub international,sfbc international,0.905229
47,jefferson parish,jefferson bancshares,0.900714
20,catapult technology,datapulse technology,0.895304
72,pinnacle marketing,pinnacle airlines,0.891690
33,first business,first united,0.889524
45,indian eyes,indiana resources,0.879465
99,watertown,waters,0.877778
83,shogun,shofu,0.875556
97,triad,trimas,0.875556


In [83]:
#threshold=0.8
df_jaro[df_jaro.score>0.87]

,0,match,score
86,spn well services,us well services,0.939951
43,hub international,sfbc international,0.905229
47,jefferson parish,jefferson bancshares,0.900714
20,catapult technology,datapulse technology,0.895304
72,pinnacle marketing,pinnacle airlines,0.891690
33,first business,first united,0.889524
45,indian eyes,indiana resources,0.879465
99,watertown,waters,0.877778
83,shogun,shofu,0.875556
97,triad,trimas,0.875556


#### 1.2  Merge Performance Evaluation
- According to the test result, the threshold for `StrSimilarity` should be larger than 0.6; the threshold for `jaro-winkler` should be larger than 0.89;
- Larger score in `StrSimilarity` provides a better sensible match, which means that we can find a clear cut-off for scores;
- For `jaro-winkler`, a larger score does not means a better sensible match: "northwestern connecticut community college" and	"northwestern" with score 0.857143, "indian eyes" and	"indiana resources" woth score 0.879465. However, the former match is more likely to be correct than the latter case;
- So the `StrSimilarity`is much stable than `jaro-winkler`

------------------------------------------------------------

#### 2. Shark Repellent —>Compustat

In [21]:
#real data test
# ccompustat is dictionary;shark repellent is query name
random.seed(123)
query_s=random.sample(s_name1,100)#1000 names 
c_name1=sorted(c_name1, key=len)    
dict_c=list(' '.join(str1.strip().translate(str.maketrans('', '', string.punctuation)).split()) for str1 in c_name1)#remove punctuation
query_s=list(' '.join(str1.strip().translate(str.maketrans('', '', string.punctuation)).split()) for str1 in query_s)
query_s.sort(reverse=False)

#### 2.1 Merge Result

- StrSimilar

In [24]:
%%time
df_s2c=name_match1(query_s,dict_c).sort_values(by='score',ascending=False)
#take longer time since the dictionary is larger

Wall time: 1min


In [33]:
df_s2c[df_s2c.score>0.5]#the propert threshold should be 0.6

,0,match,score
0,abb,abb,3.000000
63,qts realty trust,qts realty trust,3.000000
38,juniper networks,juniper networks,3.000000
42,lancaster colony,lancaster colony,3.000000
43,life time fitness,life time fitness,3.000000
...,...,...,...
48,mfs california municipal fund,mfs california municipal fd,0.714286
23,clear channel outdoor holdings,clear channel outdoor hldgs,0.697368
93,us home systems,u s home systems,0.634409
20,centuria industrial reit,dream industrial reit,0.511111


In [60]:
df_s2c[(df_s2c.score>0.6) & (df_s2c.score<2)]

,0,match,score
85,the lamson sessions,lamson sessions,1.882353
86,the malaysia fund,malaysia fund,1.866667
46,mb bancorp,bancorp,1.823529
84,the gabelli global multimedia trust,gabelli multimedia trust,1.813559
53,nuveen florida quality income municipal fund,nuveen municipal income fund,1.777778
73,silicon graphics international,silicon graphics,1.695652
64,quantum graphite,quantum,1.608696
48,mfs california municipal fund,mfs california municipal fd,0.714286
23,clear channel outdoor holdings,clear channel outdoor hldgs,0.697368
93,us home systems,u s home systems,0.634409


- Jaro-winkler

In [34]:
%%time
df_s2c_j=jaro_distance(query_s,dict_c).sort_values(by='score',ascending=False)

Wall time: 3min 51s


In [86]:
df_s2c_j[df_s2c_j.score>0.5]#the propert threshold should be 0.7424

,0,match,score
0,abb,abb,1.000000
63,qts realty trust,qts realty trust,1.000000
38,juniper networks,juniper networks,1.000000
42,lancaster colony,lancaster colony,1.000000
43,life time fitness,life time fitness,1.000000
...,...,...,...
15,bond international software,bond industries,0.832222
55,opportunity investment management,pzena investment management,0.774832
84,the gabelli global multimedia trust,gabelli multimedia trust,0.742460
77,société générale,ast general,0.742424


In [55]:
df_s2c_j[(df_s2c_j.score>0.7) & (df_s2c_j.score<0.9)]

,0,match,score
68,rision,radvision,0.900000
40,koninklijke vendex kbb,koninklijke kpn,0.899654
58,panoro energy,pan orient energy,0.893979
71,security devices international,security environmental sys,0.886778
81,tegel group holdings,thistle group holdings,0.882121
74,sirva,shiva,0.880000
35,institutional financial markets,institutional investorsdel,0.877975
94,usellcom,selco,0.875000
16,butler manufacturing,butler mfg,0.870000
83,telegraaf media groep,telemed,0.866667


#### 2.2  Merge Performance Evaluation
- According to the test result, the threshold for `StrSimilarity` should be larger than 0.5 (maybe around 0.6344); the threshold for `jaro-winkler` should be larger than 0.74;
- Larger score in `StrSimilarity` provides a better sensible match, which means that we can find a clear cut-off for scores;
- For `jaro-winkler`, a larger score does not means a better sensible match: "the gabelli global multimedia trust" and	"gabelli multimedia trust" with score 0.742460, "zenyatta ventures" and "ventures" woth score 0.833428. However,the former match is more likely to be correct than the latter case;
- So the `StrSimilarity`is much stable than `jaro-winkler`

#### 3. Burning Glass —>Compustat

In [35]:
#real data test
# ccompustat is dictionary;shark repellent is query name
random.seed(123)
query_bg=random.sample(bg_name1,100)#1000 names 
c_name1=sorted(c_name1, key=len)    
dict_c=list(' '.join(str1.strip().translate(str.maketrans('', '', string.punctuation)).split()) for str1 in c_name1)#remove punctuation
query_bg=list(' '.join(str1.strip().translate(str.maketrans('', '', string.punctuation)).split()) for str1 in query_bg)
query_bg.sort(reverse=False)

#### 3.1 Merge Result

- StrSimilar

In [61]:
%%time
df_bg2c=name_match1(query_bg,dict_c).sort_values(by='score',ascending=False)
#take longer time since the dictionary is larger

Wall time: 1min


In [62]:
df_bg2c[df_bg2c.score>0.5]#the propert threshold should be 1

,0,match,score
43,hub international,hub international,3.000000
33,first business,business first bancshs,2.777778
46,it solutions,pomeroy it solutions,2.750000
68,orbus,orbus pharma,2.588235
97,triad,triad systems,2.555556
0,1st access,access,1.750000
47,jefferson parish,jefferson,1.720000
13,banco popular,popular,1.700000
2,access security,access,1.571429
91,team sewell,team,1.533333


- Jaro Winkler

In [63]:
%%time
df_bg2c_j=jaro_distance(query_bg,dict_c).sort_values(by='score',ascending=False)

Wall time: 4min 5s


In [66]:
df_bg2c_j[df_bg2c_j.score>0.8]

,0,match,score
43,hub international,hub international,1.000000
97,triad,traid,0.946667
37,gm financial group,bmr financial group,0.946394
86,spn well services,us well services,0.939951
27,ebc,enbc,0.925000
...,...,...,...
10,arnamy,aradyme,0.826032
41,hillcrest property management,hillcrest resources ltdold,0.824481
57,massage heights franchising,mass merchandisers,0.824444
23,csu bakersfield,clearfield,0.816667


In [87]:
df_bg2c_j[df_bg2c_j.score>0.85]

,0,match,score
43,hub international,hub international,1.000000
97,triad,traid,0.946667
37,gm financial group,bmr financial group,0.946394
86,spn well services,us well services,0.939951
27,ebc,enbc,0.925000
33,first business,first busey,0.924675
46,it solutions,viq solutions,0.920940
40,hcr home care,thc homecare,0.920940
12,avivo,aviva,0.920000
2,access security,accessity,0.920000


#### 3.2  Merge Performance Evaluation
- According to the test result, the threshold for `StrSimilarity` should be larger than 0.5757 (maybe around 0.6344); the threshold for `jaro-winkler` should be larger than 0.86;
- Larger score in `StrSimilarity` provides a better sensible match, which means that we can find a clear cut-off for scores;
- For `jaro-winkler`, a larger score does not means a better sensible match: "team sewell" and	"team" with score 0.872727, "pronexus" and "pronet" woth score 0.891667.However, the former match is more likely to be correct than the latter case;
- So the `StrSimilarity`is much stable than `jaro-winkler`  

### Conclusion
1. `StrSimilarity`'s thresholds are around 0.6 across the three dataset, while the threshold for `Jaro-winkler`'s vary a lot for different data sets. 
- `StrSimilarity`'s score can be regarded as a monotone function of the probability of correct match, while `Jaro-winkler`'s not.